# Clean Data

In [22]:
from pyspark.sql.functions import to_timestamp,year, month

StatementMeta(, e2510418-89dc-473e-8229-c05d3eab672f, 24, Finished, Available)

In [23]:
path = "abfss://4155d96d-1328-43ad-982f-7aecb1ee2008@onelake.dfs.fabric.microsoft.com/bf01a96a-f669-4bd4-a676-8dbd2382d089/Files/"
filename = "iload_test_1.csv"
df = spark.read.format("csv").option("header","true").load(path + filename)




    
df = df.withColumn("order date (DateOrders)", to_timestamp("order date (DateOrders)", "M/d/yyyy H:mm"))
df = df.withColumn("shipping date (DateOrders)", to_timestamp("shipping date (DateOrders)", "M/d/yyyy H:mm"))



df.createOrReplaceTempView("df")

StatementMeta(, e2510418-89dc-473e-8229-c05d3eab672f, 25, Finished, Available)

In [24]:
nearest_df = spark.read.option("header", "true").csv("abfss://4155d96d-1328-43ad-982f-7aecb1ee2008@onelake.dfs.fabric.microsoft.com/900594c3-14cd-4919-8406-174ac04587bb/Files/orders/year=2018")
nearest_df.createOrReplaceTempView("nearest_df")

StatementMeta(, e2510418-89dc-473e-8229-c05d3eab672f, 26, Finished, Available)

In [25]:
df = spark.sql("""
    SELECT * 
    FROM df 
    WHERE `order date (DateOrders)` > ( 
        SELECT order_date_dateorders FROM
        nearest_df nd ORDER BY  order_date_dateorders DESC LIMIT 1
    )               
""")

StatementMeta(, e2510418-89dc-473e-8229-c05d3eab672f, 27, Finished, Available)

In [26]:
# Create a copy of values
dataco = df 

StatementMeta(, e2510418-89dc-473e-8229-c05d3eab672f, 28, Finished, Available)

In [27]:
#Tạo một function để tách bảng


def seperate_table(df, lst_table):
    
    new_df = df.select(*lst_table)
    return new_df

StatementMeta(, e2510418-89dc-473e-8229-c05d3eab672f, 29, Finished, Available)

# Create Orders detail table

In [28]:
#import mấy thư viện cần thiết
from pyspark.sql.functions import concat,col,lit,cast, sum


StatementMeta(, e2510418-89dc-473e-8229-c05d3eab672f, 30, Finished, Available)

In [29]:
import re

def sanitize_column_name(column_name):
    # Replace spaces with underscores
    sanitized_name = column_name.replace(' ', '_')
    # Convert to lowercase
    sanitized_name = sanitized_name.lower()
    return sanitized_name

# Get the current column names
columns = dataco.columns

# Create a dictionary to map old column names to sanitized names
renaming_dict = {col: sanitize_column_name(col) for col in columns}

# Rename columns in the DataFrame
for old_name, new_name in renaming_dict.items():
    dataco = dataco.withColumnRenamed(old_name, new_name)




StatementMeta(, e2510418-89dc-473e-8229-c05d3eab672f, 31, Finished, Available)

In [30]:
# renamed thằng type thành type of transaction cho dễ hiểu hơn
dataco = dataco.withColumnRenamed('Type','type_of_transaction')


StatementMeta(, e2510418-89dc-473e-8229-c05d3eab672f, 32, Finished, Available)

In [31]:
def change_type(data, columns, types):
    for col in columns:
        data  = data.withColumn(col, data[col].cast(types))

    return data

columns_to_change_int = ["days_for_shipping_(real)","days_for_shipment_(scheduled)","Late_delivery_risk","order_item_quantity"]
dataco = change_type(dataco,columns_to_change_int, "int")  


columns_to_change_floats = ["benefit_per_order", "sales_per_customer", "latitude", "longitude",
 "order_item_discount", "order_item_discount_rate", "order_item_product_price",
 "order_item_profit_ratio", "sales", "order_profit_per_order", "product_price"]


dataco = change_type(dataco, columns_to_change_floats,"float")

StatementMeta(, e2510418-89dc-473e-8229-c05d3eab672f, 33, Finished, Available)

In [32]:
# lấy những thông tin cần cho bảng orders sau đó group nó lại 

columns_order_table = ['order_id', 'customer_id','product_card_id' ,'order_date_(dateorders)', 
'shipping_date_(dateorders)', 'shipping_mode', 'delivery_status', 
'type_of_transaction', 'late_delivery_risk', 'order_status', 'market', 
'order_region', 'order_country', 'order_state', 'order_city', 'customer_country', 
'customer_city', 'customer_street', 'sales', 'benefit_per_order', 'order_item_discount',
'order_item_quantity', 'order_profit_per_order', "longitude", "latitude"]
order_details = seperate_table(dataco, columns_order_table)

StatementMeta(, e2510418-89dc-473e-8229-c05d3eab672f, 34, Finished, Available)

In [33]:
from pyspark.sql.functions import sum as spark_sum
import pyspark.sql.functions as F

from pyspark.sql.functions import concat_ws,when

StatementMeta(, e2510418-89dc-473e-8229-c05d3eab672f, 35, Finished, Available)

In [34]:
from pyspark.sql.functions import monotonically_increasing_id
from pyspark.sql import functions as F
from pyspark.sql.window import Window
from pyspark.sql.functions import filter

from pyspark.sql.functions import row_number

StatementMeta(, e2510418-89dc-473e-8229-c05d3eab672f, 36, Finished, Available)

In [35]:
order_details = order_details.withColumn("concat_territory", concat_ws("_", col("market"), col("order_region"), col("order_country"),col("order_state"), col("order_city")))


# làm tương tự như thằng territory
order_details = order_details.withColumn("concat_store_address", concat_ws("_", col("customer_country"), col("customer_city"),col("customer_street")))

StatementMeta(, e2510418-89dc-473e-8229-c05d3eab672f, 37, Finished, Available)

In [50]:
old_territory = spark.read.option("header", "true").csv("abfss://4155d96d-1328-43ad-982f-7aecb1ee2008@onelake.dfs.fabric.microsoft.com/900594c3-14cd-4919-8406-174ac04587bb/Files/territory")

old_territory.createOrReplaceTempView("old_territory")

StatementMeta(, e2510418-89dc-473e-8229-c05d3eab672f, 52, Finished, Available)

# Create territory table

In [51]:
#tạo bảng territory
territory = order_details.select("concat_territory","market", "order_region","order_country", "order_state", "order_city")
# drop những giá trị trùng lặp từ bảng

territory = territory.dropDuplicates(subset=["concat_territory","market", "order_region","order_country", "order_state", "order_city"])

# rename để tránh trường hợp  ambitigous khi join 
territory  = territory.withColumnRenamed("concat_territory" , "ter_concat_territory")

territory.createOrReplaceTempView("territory")
territory = spark.sql("""
SELECT * FROM territory t WHERE t.ter_concat_territory NOT IN (
    SELECT ter_concat_territory FROM old_territory
)

""")

StatementMeta(, e2510418-89dc-473e-8229-c05d3eab672f, 53, Finished, Available)

In [54]:
display(territory)

StatementMeta(, e2510418-89dc-473e-8229-c05d3eab672f, 56, Finished, Available)

SynapseWidget(Synapse.DataFrame, 88f31678-8492-4007-b442-8f03c0280efd)

# Create store table 

In [ ]:
old_store = spark.read.option("header", "true").csv("abfss://4155d96d-1328-43ad-982f-7aecb1ee2008@onelake.dfs.fabric.microsoft.com/900594c3-14cd-4919-8406-174ac04587bb/Files/store")
old_store.createOrReplaceTempView("old_store")


StatementMeta(, , , Cancelled, )

In [ ]:

columns_store_table = ["concat_store_address", "customer_country" , "customer_city" , "customer_street", "latitude", "longitude" ]

store_table = order_details.select(columns_store_table)

store_table = store_table.dropDuplicates(subset=["concat_store_address", "customer_country" , "customer_city" , "customer_street"])

store_table = store_table.withColumnRenamed("concat_store_address" , "st_concat_store_address")
store_table.createOrReplaceTempView("store")
# kiểm tra xem nó có bao nhiêu dòng
store_table = spark.sql("""
SELECT * FROM store WHERE st_concat_store_address NOT IN(
    SELECT st_concat_store_address FROM old_store
)

""")

StatementMeta(, , , Cancelled, )

In [ ]:
duplicates_store = store_table.groupBy("st_concat_store_address").count().filter(col("count") > 1)
duplicates_store.show()

StatementMeta(, , , Cancelled, )

In [ ]:
store_column_renamed = {
    "customer_country" : "store_country" ,
    "customer_city" : "store_city" , 
    "customer_street" : "store_street"
}

for old_name, new_name in store_column_renamed.items():
    store_table = store_table.withColumnRenamed(old_name,new_name)

StatementMeta(, , , Cancelled, )

In [ ]:
store_table.show(5)

StatementMeta(, , , Cancelled, )

In [ ]:
order_details = order_details.drop("market","order_region","order_country","order_state",  "order_city", "customer_country", "customer_city", "customer_street", "longitude", "latitude")


StatementMeta(, , , Cancelled, )

# Create product table

In [ ]:
old_product =spark.read.format("csv").option("header", "true").load("abfss://4155d96d-1328-43ad-982f-7aecb1ee2008@onelake.dfs.fabric.microsoft.com/900594c3-14cd-4919-8406-174ac04587bb/Files/product")

StatementMeta(, , , Cancelled, )

In [ ]:
lst_product = ["product_card_id" , "product_name", "product_price", "product_status", "category_id"]


product = seperate_table(dataco,lst_product)

product = product.dropDuplicates(subset = lst_product)
product.createOrReplaceTempView("product")

product = product.exceptAll(old_product)

StatementMeta(, , , Cancelled, )

In [ ]:
display(product)

StatementMeta(, , , Cancelled, )

# Create Category table

In [ ]:
old_category = spark.read.format("csv").option("header", "true").load("abfss://4155d96d-1328-43ad-982f-7aecb1ee2008@onelake.dfs.fabric.microsoft.com/900594c3-14cd-4919-8406-174ac04587bb/Files/category")

StatementMeta(, , , Cancelled, )

In [ ]:
lst_category = ["category_id", "category_name", "department_id"]

category  = seperate_table(dataco, lst_category)
category = category.dropDuplicates(subset= lst_category)

category = category.exceptAll(old_category)

display(category)



StatementMeta(, , , Cancelled, )

# Create Department table

In [ ]:
old_department = spark.read.format("csv").option("header", "true").load("abfss://4155d96d-1328-43ad-982f-7aecb1ee2008@onelake.dfs.fabric.microsoft.com/900594c3-14cd-4919-8406-174ac04587bb/Files/department")

StatementMeta(, , , Cancelled, )

In [ ]:
lst_department = ["department_id", "department_name"]

department  = seperate_table(dataco,lst_department)


department = department.dropDuplicates(subset=lst_department)

department = department.exceptAll(old_department)
display(department)



StatementMeta(, , , Cancelled, )

# Create Transaction Table

In [ ]:
old_transaction = spark.read.format("csv").option("header", "true").load("abfss://4155d96d-1328-43ad-982f-7aecb1ee2008@onelake.dfs.fabric.microsoft.com/900594c3-14cd-4919-8406-174ac04587bb/Files/transaction")

StatementMeta(, , , Cancelled, )

In [ ]:

transaction = seperate_table(dataco, ["type_of_transaction"])
transaction = transaction.dropDuplicates(subset = ["type_of_transaction"])
transaction = transaction.exceptAll(old_transaction)
transaction.show()

StatementMeta(, , , Cancelled, )

# Create Shipping Mode table

In [ ]:
old_shipping_mode = old_transaction = spark.read.format("csv").option("header", "true").load("abfss://4155d96d-1328-43ad-982f-7aecb1ee2008@onelake.dfs.fabric.microsoft.com/900594c3-14cd-4919-8406-174ac04587bb/Files/shipping_mode")

StatementMeta(, , , Cancelled, )

In [ ]:
shipping_mode = seperate_table(dataco, ["shipping_mode"])
shipping_mode = shipping_mode.dropDuplicates(subset = ["shipping_mode"])

shipping_mode = shipping_mode.exceptAll(old_shipping_mode)
shipping_mode.show()

StatementMeta(, , , Cancelled, )

# Create order status

In [ ]:
old_order_status = old_transaction = spark.read.format("csv").option("header", "true").load("abfss://4155d96d-1328-43ad-982f-7aecb1ee2008@onelake.dfs.fabric.microsoft.com/900594c3-14cd-4919-8406-174ac04587bb/Files/order_status")

StatementMeta(, , , Cancelled, )

In [ ]:
order_status = seperate_table(dataco , ["order_status"])
order_status = order_status.dropDuplicates(["order_status"])
order_status = order_status.exceptAll(old_order_status)

order_status.show()

StatementMeta(, , , Cancelled, )

# Create Customer Table 

In [ ]:
old_customer = old_transaction = spark.read.format("csv").option("header", "true").load("abfss://4155d96d-1328-43ad-982f-7aecb1ee2008@onelake.dfs.fabric.microsoft.com/900594c3-14cd-4919-8406-174ac04587bb/Files/customer")

In [ ]:
lst_customers = ["customer_id", "customer_fname", "customer_lname", "customer_segment"]


customers = seperate_table(dataco, lst_customers)
customers = customers.dropDuplicates(subset = lst_customers)
customers = customers.withColumn("customer_fullname", concat(col("customer_fname"), lit(" "), col("customer_lname")))
customers = customers.drop("customer_fname", "customer_lname")

customer = customers.exceptAll(old_customer)
customers.show()


# Load to silver

In [ ]:
order_renamed = {
    "days_for_shipping_(real)" : "days_for_shipping_real",
    "days_for_shipment_(scheduled)": "days_for_shipment_scheduled",
    "shipping_date_(dateorders)" : "shipping_date_dateorders" ,
    "order_date_(dateorders)" : "order_date_dateorders"
     

}
for old_name , new_name in order_renamed.items():
    order_details = order_details.withColumnRenamed(old_name,new_name)

## Load order details

In [ ]:
# tao partition de phan du lieu theo year month
order_details = order_details.withColumn("year", year('order_date_dateorders'))
order_details = order_details.withColumn("month", month('order_date_dateorders'))


order_details.write \
    .mode("append") \
    .partitionBy("year", "month") \
    .format("csv") \
    .option("header", "true") \
    .save("Files/orders")

## Load Product

In [ ]:
product.write \
    .mode("append") \
    .format("csv") \
    .option("header", "true") \
    .save("Files/product")

## Load Category

In [ ]:
category.write \
    .mode("append") \
    .format("csv") \
    .option("header", "true") \
    .save("Files/category")

## Load Department

In [ ]:
department.write \
    .mode("append") \
    .format("csv") \
    .option("header", "true") \
    .save("Files/department")

StatementMeta(, , , Cancelled, )

## Load territory

In [ ]:
territory.write \
    .mode("append") \
    .format("csv") \
    .option("header", "true") \
    .save("Files/territory")

StatementMeta(, , , Cancelled, )

## Load store region 

In [ ]:
store_table.write \
    .mode("append") \
    .format("csv") \
    .option("header", "true") \
    .save("Files/store")

StatementMeta(, , , Cancelled, )

## Load Customers

In [ ]:
customers.write \
    .mode("append") \
    .format("csv") \
    .option("header", "true") \
    .save("Files/customer")

StatementMeta(, , , Cancelled, )

## load transaction

In [ ]:
transaction.write \
    .mode("append") \
    .format("csv") \
    .option("header", "true") \
    .save("Files/transaction")

StatementMeta(, , , Cancelled, )

## Load shipping mode

In [ ]:
shipping_mode.write \
    .mode("append") \
    .format("csv") \
    .option("header", "true") \
    .save("Files/shipping_mode")

StatementMeta(, , , Cancelled, )

## Load order status

In [ ]:
order_status.write \
    .mode("append") \
    .format("csv") \
    .option("header", "true") \
    .save("Files/order_status")

StatementMeta(, , , Cancelled, )